In [2]:
import pandas as pd
import glob
import os

In [ ]:
#pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Five files of biomarker data to be merged ans features selected here. 


Author: Neha Sindhu 


In [3]:

path = os.path.expanduser("~/Documents/BINF/competition/Pre_Processed_datasets")

In [4]:
#Get .pkl files
pkl_files = glob.glob(os.path.join(path, "*.pkl"))
#Load them as df
dataframes = [pd.read_pickle(f) for f in pkl_files]

In [5]:
#Create a dict keyed by filename
df_dict = {os.path.basename(f).replace(".pkl", ""): pd.read_pickle(f) for f in pkl_files}

In [ ]:
#concatenate them into one huge dataframe
big_df = pd.concat(
    [df.assign(source=name) for name, df in df_dict.items()],
    ignore_index=True,
    sort=False
)

# sanity check: should be 25 if 5×5
print(len(big_df))

1326


In [7]:
big_df.head()

,Participant ID_clinical_df1,Tissue Source_df1,Protocol_df1,Sample Type_clinical_df1,Enrollment Category_df1,Primary Adjudicated Category_df1,Sex_df1,Age (Years) (Binned)_df1,Race_df1,KDIGO Stage_df1,...,RACE_df2,SAMPLE_AMOUNT_df2,SAMPLE_AMOUNT_UNITS_df2,SAMPLE_BOX_LOCATION_df2,VOLUME_EXTRACTED,Participant ID_OP-UHPLC MS__df2,Sample Type_OP-UHPLC MS__df2,Study type_df2,Biopsy Time Interval Days Post Consent,Sample Collection Interval Days Post Consent
0,27-10039,KPMP Recruitment Site,KPMP Main Protocol,Percutaneous Needle Biopsy,CKD,Diabetic Kidney Disease,Female,60-69 Years,White,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,27-10039,KPMP Recruitment Site,KPMP Main Protocol,Percutaneous Needle Biopsy,CKD,Diabetic Kidney Disease,Female,60-69 Years,White,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27-10041,KPMP Recruitment Site,KPMP Main Protocol,Percutaneous Needle Biopsy,CKD,Diabetic Kidney Disease,Female,60-69 Years,Asian,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27-10041,KPMP Recruitment Site,KPMP Main Protocol,Percutaneous Needle Biopsy,CKD,Diabetic Kidney Disease,Female,60-69 Years,Asian,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27-10042,KPMP Recruitment Site,KPMP Main Protocol,Percutaneous Needle Biopsy,CKD,Diabetic Kidney Disease,Female,50-59 Years,White,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:

#save big_df and the column names

# Save column names
# UTF-8 text file for column names
with open(os.path.join(path, "big_df_columns.txt"), "w", encoding="utf-8", newline="\n") as f:
    for col in big_df.columns:
        f.write(f"{col}\n")

#Save full dataframe
big_df.to_excel(os.path.join(path, "big_df.xlsx"), index = False)

In [16]:
#find and delete the columns where all rows had the same entry. 
# write those column names into a txt file

# find columns with the same value in all rows
constant_cols = [col for col in big_df.columns if big_df[col].nunique() == 1]
print(len(constant_cols))

os.makedirs(path, exist_ok=True)
# save their names

with open(os.path.join(path, "constant_columns.txt"), "w", encoding="utf-8") as f:
    for col in constant_cols:
        f.write(f"{col}\n")


# drop those columns from the dataframe
big_df = big_df.drop(columns=constant_cols)

#save the cleaned dataframe
big_df.to_excel(os.path.join(path, "big_df1.xlsx"), index=False)

43


In [17]:
#delete column containing the types of participant visit
cols_to_drop = [col for col in big_df.columns if col.startswith("Participant Visit")]
big_df = big_df.drop(columns=cols_to_drop)

In [19]:
#view the df size

print(f"Rows: {big_df.shape[0]}, Columns: {big_df.shape[1]}")

Rows: 1326, Columns: 191


In [20]:
#delete every column starting with 'Participant' except the column Participant_ID_norm

if big_df["Participant_ID_norm"].isna().sum() == 0:
    cols_to_drop = [col for col in big_df.columns
                    if col.startswith("Participant") and col != "Participant_ID_norm"]
    big_df = big_df.drop(columns=cols_to_drop)


In [21]:
print(f"Rows: {big_df.shape[0]}, Columns: {big_df.shape[1]}")

Rows: 1326, Columns: 181


In [24]:
cols_to_drop = [col for col in big_df.columns if "Sample" in col or "SAMPLE" in col]
big_df = big_df.drop(columns=cols_to_drop)

In [25]:
print(f"Rows: {big_df.shape[0]}, Columns: {big_df.shape[1]}")

Rows: 1326, Columns: 162


In [29]:
#are columns Biopsy Time Interval Days Post Consent_df1 and Biopsy Time Interval Days Post Consent_df2 identical?
(big_df["Biopsy Time Interval Days Post Consent_df1"]
 .equals(big_df["Biopsy Time Interval Days Post Consent_df2"]))

True

In [30]:
#big_df = big_df.drop(columns=["Biopsy Time Interval Days Post Consent"])
big_df["Days_Post_Biopsy"] = big_df["Biopsy Time Interval Days Post Consent_df1"].combine_first(
    big_df["Biopsy Time Interval Days Post Consent_df2"]
)
big_df = big_df.drop(columns=[
    "Biopsy Time Interval Days Post Consent_df1",
    "Biopsy Time Interval Days Post Consent_df2"
])

C:\Users\neha\AppData\Local\Temp\ipykernel_3648\1748210524.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  big_df["Days_Post_Biopsy"] = big_df["Biopsy Time Interval Days Post Consent_df1"].combine_first(


In [36]:
big_df["Biopsy Time Interval Days Post Consent"].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: Biopsy Time Interval Days Post Consent, dtype: float64

In [ ]:
#are there rows where there are non NaN values in both columns Days_Post_Biosy and Biopsy Time Interval Days Post Consent
mask = big_df["Days_Post_Biopsy"].notna() & big_df["Biopsy Time Interval Days Post Consent"].notna()
mask.any()

np.False_

In [ ]:
#merge the columns Days_Post_Biosy and Biopsy Time Interval Days Post Consent. the entry of the resulting columns shouls be whatever the value was on the non-nan values in the original columns.
#  The resulting column are named Days_after_biopsy
big_df["Days_after_biopsy"] = big_df["Days_Post_Biopsy"].combine_first(
    big_df["Biopsy Time Interval Days Post Consent"]
)
big_df = big_df.drop(columns=[
    "Days_Post_Biopsy",
    "Biopsy Time Interval Days Post Consent"
])

In [40]:

big_df["Days_after_biopsy"].isna().sum()


np.int64(118)

In [43]:
#For all rows whose days post biopsy is NaN, replace it with -1 (to replace later)
big_df["Days_after_biopsy"] = big_df["Days_after_biopsy"].fillna(-1)

In [44]:
#for every set of rows with the same Participant_ID_norm but different Days_after_Biopsy, 
# delete all rows except the one whose Days_after_Biopsy is the smallest positive integer.
import numpy as np
import pandas as pd

# pick the correct column name
dab_col = "Days_after_biopsy" if "Days_after_biopsy" in big_df.columns else "Days_after_Biopsy"

# ensure numeric
big_df[dab_col] = pd.to_numeric(big_df[dab_col], errors="coerce")

# positive integers only
pos_int = (big_df[dab_col] > 0) & ((big_df[dab_col] % 1) == 0)

# smallest positive integer per participant
idx_keep_pos = (big_df[pos_int]
                .groupby("Participant_ID_norm")[dab_col]
                .idxmin())

# participants with no positive DAB
all_pids = big_df["Participant_ID_norm"].unique()
pids_with_pos = big_df.loc[pos_int, "Participant_ID_norm"].unique()
pids_no_pos = set(all_pids) - set(pids_with_pos)

idx_keep_no = []
for pid in pids_no_pos:
    grp = big_df[big_df["Participant_ID_norm"] == pid]
    cand = grp.index[grp[dab_col] == -1]
    if len(cand):
        idx_keep_no.append(cand[0])
        print("row has no DAB")  # one print per such participant
    else:
        # fallback: keep the first row if -1 is missing (should be rare)
        idx_keep_no.append(grp.index[0])
        print("row has no DAB")

keep_idx = pd.Index(idx_keep_pos.tolist() + idx_keep_no)
big_df = big_df.loc[keep_idx].copy()


row has no DAB
row has no DAB
row has no DAB


In [48]:
print(f"Rows: {big_df.shape[0]}, Columns: {big_df.shape[1]}")

Rows: 207, Columns: 160


In [46]:
#For all rows whose days post biopsy is -1, replace it with 0
big_df["Days_after_biopsy"] = big_df["Days_after_biopsy"].replace(-1, 0)

In [47]:
#check if Participant_ID_norm is unique
big_df["Participant_ID_norm"].is_unique

True

Now we have 207 patients' data from their earliest possible time point

In [49]:
#save big_df and the column names

# Save column names
# UTF-8 text file for column names
with open(os.path.join(path, "uniquePatient_columns.txt"), "w", encoding="utf-8", newline="\n") as f:
    for col in big_df.columns:
        f.write(f"{col}\n")

#Save full dataframe
big_df.to_excel(os.path.join(path, "uniquePatient.xlsx"), index = False)

In [51]:
#delete all columns without at least 10 non-NaN values

big_df = big_df.dropna(axis=1, thresh=10)

In [ ]:
#drop all columns whose non-NA values are all identical
big_df = big_df.drop(columns=[col for col in big_df.columns if big_df[col].nunique(dropna=True) == 1])

In [53]:
print(f"Rows: {big_df.shape[0]}, Columns: {big_df.shape[1]}")

Rows: 207, Columns: 66


In [54]:
#save big_df and the column names

# Save column names
# UTF-8 text file for column names
with open(os.path.join(path, "uniquePatient_columns2.txt"), "w", encoding="utf-8", newline="\n") as f:
    for col in big_df.columns:
        f.write(f"{col}\n")

#Save full dataframe
big_df.to_excel(os.path.join(path, "uniquePatient2.xlsx"), index = False)

In [55]:
#check if there are pairs of column that have exactly the same entries. 
# If yes, merge them, and rename them with the name of the first of the column

# identify sets of columns with identical entries (NaNs in the same rows count as identical)
col_hash = {
    col: pd.util.hash_pandas_object(big_df[col], index=False).values.tobytes()
    for col in big_df.columns
}

groups = {}
for col, h in col_hash.items():
    groups.setdefault(h, []).append(col)

# columns to drop (keep the first name in each identical set)
to_drop = []
merged_pairs = []
for cols in groups.values():
    if len(cols) > 1:
        merged_pairs.append((cols[0], cols[1:]))  # kept, dropped
        to_drop.extend(cols[1:])

# drop duplicate-identical columns
if to_drop:
    big_df = big_df.drop(columns=to_drop)

# optional: view what was merged
for kept, dropped in merged_pairs:
    print(f"merged into '{kept}': {', '.join(dropped)}")






merged into 'Primary Adjudicated Category_df1': Primary Adjudicated Category_df2
merged into 'Sex_df1': Sex_df2
merged into 'Age (Years) (Binned)_df1': Age (Years) (Binned)_df2
merged into 'Race_df1': Race_df2
merged into 'Baseline eGFR (ml/min/1.73m2)_df1': Baseline eGFR (ml/min/1.73m2)_df2
merged into 'Baseline eGFR (ml/min/1.73m2) (Binned)_df1': Baseline eGFR (ml/min/1.73m2) (Binned)_df2
merged into 'Proteinuria (mg) (Binned)_df1': Proteinuria (mg) (Binned)_df2
merged into 'A1c (%) (Binned)_df1': A1c (%) (Binned)_df2
merged into 'Albuminuria (mg) (Binned)_df1': Albuminuria (mg) (Binned)_df2
merged into 'Diabetes History_df1': Diabetes History_df2
merged into 'Diabetes Duration (Years)_df1': Diabetes Duration (Years)_df2
merged into 'Hypertension History_df1': Hypertension History_df2
merged into 'Hypertension Duration (Years)_df1': Hypertension Duration (Years)_df2
merged into 'On RAAS Blockade_df1': On RAAS Blockade_df2


In [56]:
print(f"Rows: {big_df.shape[0]}, Columns: {big_df.shape[1]}")

Rows: 207, Columns: 52


In [57]:
#save big_df and the column names

# Save column names
# UTF-8 text file for column names
with open(os.path.join(path, "uniquePatient_columns3.txt"), "w", encoding="utf-8", newline="\n") as f:
    for col in big_df.columns:
        f.write(f"{col}\n")

#Save full dataframe
big_df.to_excel(os.path.join(path, "uniquePatient3.xlsx"), index = False)

In [59]:
#The heme, lipemic, and icteric indices are sample quality control metrics, not biological variables.
#remove rows with too high values for these: per-site z-scores, applies absolute cutoffs.
#then remove the columns

# Define thresholds
HI_COL = "Heme index"      
LI_COL = "Lipemic index"   
II_COL = "Icteric index"   

# ---- ABSOLUTE EXCLUSION THRESHOLDS ----
HI_MAX = 100.0   # > 100 → exclude (gross hemolysis)
LI_MAX = 200.0   # > 200 → exclude (gross lipemia)
II_MAX = 10.0    # > 10  → exclude (gross icterus)

# ---- APPLY QC FILTER ----
df = big_df.copy()

# Ensure numeric
for c in [HI_COL, LI_COL, II_COL]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Identify threshold failures
df["fail_heme"]   = df[HI_COL] > HI_MAX
df["fail_lipemic"] = df[LI_COL] > LI_MAX
df["fail_icteric"] = df[II_COL] > II_MAX

# Combine
df["exclude"] = df[["fail_heme", "fail_lipemic", "fail_icteric"]].any(axis=1)

# Record reason(s)
def get_reason(row):
    reasons = []
    if row["fail_heme"]:
        reasons.append(f"{HI_COL}>{HI_MAX}")
    if row["fail_lipemic"]:
        reasons.append(f"{LI_COL}>{LI_MAX}")
    if row["fail_icteric"]:
        reasons.append(f"{II_COL}>{II_MAX}")
    return ", ".join(reasons)

df["exclude_reason"] = df.apply(get_reason, axis=1)

# Split outputs
filtered_df = df.loc[~df["exclude"]].copy()
exclusions_df = df.loc[df["exclude"], 
                       [HI_COL, LI_COL, II_COL, "exclude_reason"]].copy()

# QC summary
qc_report = {
    "n_total": int(len(df)),
    "n_excluded": int(df["exclude"].sum()),
    "pct_excluded": float(100 * df["exclude"].mean()),
}

print(qc_report)
filtered_df.head()
exclusions_df.head()


{'n_total': 207, 'n_excluded': 0, 'pct_excluded': 0.0}


,Heme index,Lipemic index,Icteric index,exclude_reason


All indices are within threshold.

In [60]:
#remove the columns of the three QC indices

big_df = big_df.drop(columns=["Heme index", "Lipemic index", "Icteric index"])

In [61]:
print(f"Rows: {big_df.shape[0]}, Columns: {big_df.shape[1]}")

Rows: 207, Columns: 49


In [64]:
#drop rows that are relevant to the protocol, but not meaningful model inputs

big_df = big_df.drop(columns=["Total volume (L)", "Total hours"])

In [65]:
#drop rows that should not be used

##source (dataset origin: confounding risk)
##Primary Adjudicated Category_df1 (too close to outcome, risk of leakage)
##Days_after_biopsy (time index, not intrinsic biology)

big_df = big_df.drop(columns=["source", "Primary Adjudicated Category_df1","Days_after_biopsy" ])

In [66]:
print(f"Rows: {big_df.shape[0]}, Columns: {big_df.shape[1]}")

Rows: 207, Columns: 44


In [67]:
#save big_df and the column names

# Save column names
# UTF-8 text file for column names
with open(os.path.join(path, "uniquePatient_columns4.txt"), "w", encoding="utf-8", newline="\n") as f:
    for col in big_df.columns:
        f.write(f"{col}\n")

#Save full dataframe
big_df.to_excel(os.path.join(path, "uniquePatient4.xlsx"), index = False)

In [ ]:
#2. If the same variable is given as absolute value in one column, and binned in another, then the binned column is removed
#merge the columns that indicate the same things, but have different column names
big_df["Creatinine_Index_mgkgday"] = big_df["Creatinine Index (mg/kg/day)"].combine_first(
    big_df["Creatinine Index(mg/kg/day)"]
)
big_df = big_df.drop(columns=["Creatinine Index (mg/kg/day)", "Creatinine Index(mg/kg/day)", "Baseline eGFR (ml/min/1.73m2) (Binned)_df1"])






In [69]:
#save big_df and the column names

# Save column names
# UTF-8 text file for column names
with open(os.path.join(path, "uniquePatient_columns5.txt"), "w", encoding="utf-8", newline="\n") as f:
    for col in big_df.columns:
        f.write(f"{col}\n")

#Save full dataframe
big_df.to_excel(os.path.join(path, "uniquePatient5.xlsx"), index = False)

On the downloaded data: 
1. changed column names to be more readable (removed _df etc) 


In [70]:
print(f"Rows: {big_df.shape[0]}, Columns: {big_df.shape[1]}")

Rows: 207, Columns: 42
